**Descripción de la dinámica del sistema depredador-presa con ruido aditivo:**

**Trayectoria en el plano de fase (panel izquierdo)**

Eje horizontal (x): Representa la población de **presas 𝑅**

Eje vertical (y): Representa la población de **depredadores 𝑁**

La línea roja con marcadores indica cómo va evolucionando el estado (𝑅,𝑁) en cada iteración. Cada punto es el resultado de integrar el sistema durante un intervalo de tiempo y luego añadir una pequeña perturbación aleatoria (ruido).

Se observa que los puntos oscilan alrededor de un valor central (cerca de la intersección de las líneas rojas y azules en los otros gráficos), que corresponde aproximadamente al estado de equilibrio.

Diagrama de fase que muestra la relación entre las dos variables del sistema **(presa 𝑅 en el eje horizontal y depredador 𝑁 en el eje vertical)** conforme avanza la simulación. A diferencia de un diagrama de fase continuo (donde se traza una curva suave a lo largo del tiempo), aquí:

*Se integra el sistema desde un estado inicial (o el estado al final de la iteración anterior) durante un intervalo de tiempo 𝑇.

*Al final de esa integración, se añade un ruido gaussiano a ambas variables.

Se registra el nuevo estado (𝑅,𝑁) y se pasa a la siguiente iteración, repitiendo el proceso.

"Cada punto rojo corresponde al estado final de una iteración, y la línea roja que los une indica el orden en que se generaron (como si fuera un “camino” que va saltando de un punto a otro). Debido a la adición de ruido en cada paso, el sistema no sigue una trayectoria suave, sino que zigzaguea alrededor del estado de equilibrio (aproximadamente en el centro del gráfico)".

**Distribución de 𝑅 (presa) (panel central)**

Es un histograma de los valores finales de 𝑅 en cada una de las iteraciones.

El eje horizontal indica el valor de 𝑅. El eje vertical muestra la frecuencia (cuántas veces aparece cada rango de valores).

La línea vertical roja indica el valor de equilibrio teórico de 𝑅.

Se aprecia que, debido al ruido, los valores de 

𝑅 se distribuyen alrededor de este equilibrio, pero no se quedan fijos en un único valor.

**Distribución de 𝑁 (depredador) (panel derecho)**

Análogo al histograma anterior, pero para la variable 𝑁.

El eje horizontal indica el valor de 𝑁. El eje vertical muestra la frecuencia.

La línea vertical azul señala el valor de equilibrio de 𝑁.

Nuevamente, se ve que los valores de 𝑁 fluctúan alrededor de ese equilibrio.
En conjunto, los tres gráficos muestran cómo, al simular iterativamente el sistema y añadir ruido en cada paso, los valores de presa y depredador no se mantienen exactamente en su equilibrio, sino que oscilan alrededor de él, generando una distribución de posibles valores finales para cada variable.

In [ ]:
# Importar librerías necesarias
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import warnings
warnings.filterwarnings("ignore")

# Definición del modelo depredador-presa con término de ruido aditivo
def predator_prey(t, y, r, K, a, c, delta):
    R, N = y
    dRdt = r * R * (1 - R / K) - a * R * N
    dNdt = c * a * R * N - delta * N
    return [dRdt, dNdt]

# Función para correr una simulación iterativa
def run_simulation(r=1, K=1, a=1, c=1, delta=0.5, noise_std=0.01, T=1.0, iterations=50):
    # Calcular el estado de equilibrio (para condiciones no triviales)
    # Para la segunda ecuación (depredador) se tiene: c*a*R - delta = 0  => R_eq = delta/(c*a)
    # Y para la presa, usando la primera ecuación:
    #   r*R*(1-R/K) = a*R*N  => N_eq = r*(1-R_eq/K)/a, si R_eq < K; en caso contrario N_eq = 0.
    if a == 0 or c == 0:
        equilibrium = np.array([1.0, 0.01])
    else:
        R_eq = delta / (c * a)
        N_eq = r * (1 - R_eq / K) / a if R_eq < K else 0
        equilibrium = np.array([R_eq, N_eq])
    
    state = equilibrium.copy()  # iniciamos en el equilibrio
    states = np.zeros((iterations, 2))
    timespan = [0, T]
    
    # En cada iteración integramos el sistema por un lapso T y luego se añade ruido gaussiano
    for i in range(iterations):
        sol = solve_ivp(predator_prey, timespan, state, args=(r, K, a, c, delta), dense_output=True)
        state_det = sol.y[:, -1]  # estado al final del intervalo de integración
        noise = np.random.normal(0, noise_std, size=2)  # ruido aditivo para cada variable
        state = state_det + noise
        states[i, :] = state
    return states, equilibrium

# Función que genera los gráficos interactivos
def plot_simulation(r=1, K=1, a=1, c=1, delta=0.5, noise_std=0.01, T=1.0, iterations=50):
    states, equilibrium = run_simulation(r, K, a, c, delta, noise_std, T, iterations)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Diagrama de fase: trayectoria en el plano (R, N)
    axes[0].plot(states[:, 0], states[:, 1], '-o', color='red', lw=2)
    axes[0].set_xlabel('R (Presa)')
    axes[0].set_ylabel('N (Depredador)')
    axes[0].set_title('Trayectoria en el plano de fase')
    axes[0].grid(True)
    
    # Histograma de la variable R (presa)
    axes[1].hist(states[:, 0], bins=10, color='blue', alpha=0.7)
    axes[1].axvline(equilibrium[0], color='red', lw=2, label='Equilibrio')
    axes[1].set_title('Distribución de R (presa)')
    axes[1].set_xlabel('R')
    axes[1].set_ylabel('Frecuencia')
    axes[1].legend()
    
    # Histograma de la variable N (depredador)
    axes[2].hist(states[:, 1], bins=10, color='red', alpha=0.7)
    axes[2].axvline(equilibrium[1], color='blue', lw=2, label='Equilibrio')
    axes[2].set_title('Distribución de N (depredador)')
    axes[2].set_xlabel('N')
    axes[2].set_ylabel('Frecuencia')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()

# Uso de ipywidgets para crear sliders interactivos
interact(
    plot_simulation,
    r=FloatSlider(value=1, min=0.1, max=2, step=0.1, description='r'),
    K=FloatSlider(value=1, min=0.1, max=5, step=0.1, description='K'),
    a=FloatSlider(value=1, min=0.1, max=2, step=0.1, description='a'),
    c=FloatSlider(value=1, min=0.1, max=2, step=0.1, description='c'),
    delta=FloatSlider(value=0.5, min=0.1, max=2, step=0.1, description='delta'),
    noise_std=FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='Noise sd'),
    T=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='T (intervalo)'),
    iterations=IntSlider(value=50, min=10, max=200, step=10, description='Iteraciones')
)


interactive(children=(FloatSlider(value=1.0, description='r', max=2.0, min=0.1), FloatSlider(value=1.0, descri…

<function __main__.plot_simulation(r=1, K=1, a=1, c=1, delta=0.5, noise_std=0.01, T=1.0, iterations=50)>